In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-CSE-CIC-IDS2018-v2.parquet"
data = pd.read_parquet(file_name)

In [3]:
##ADDED
file_name_test = "cleaned_cse_output.csv"
test_data = pd.read_csv(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    16635567
1     2258141
Name: count, dtype: int64

In [5]:
##ADDED
test_data.Label.value_counts()
##

Label
1    19011
0     8075
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
##ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
##ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['SSH-Bruteforce', 'Benign', 'DDoS attacks-LOIC-HTTP',
       'DDOS attack-HOIC', 'DoS attacks-Slowloris', 'DoS attacks-Hulk',
       'FTP-BruteForce', 'Infilteration', 'Bot', 'DoS attacks-GoldenEye',
       'Brute Force -Web', 'DoS attacks-SlowHTTPTest', 'SQL Injection',
       'DDOS attack-LOIC-UDP', 'Brute Force -XSS'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['SSH-Bruteforce', 'FTP-BruteForce', 'SQL Injection',
       'Brute Force -Web', 'Brute Force -XSS', 'Benign', 'Infilteration',
       'DoS attacks-Slowloris', 'DoS attacks-SlowHTTPTest',
       'DoS attacks-Hulk'], dtype=object)

In [12]:
data = data.groupby(by='Attack').sample(frac=0.02, random_state=13)

In [13]:
##ADDED
# test_data = test_data.groupby(by='Attack').sample(frac=0.02, random_state=13)
##

In [14]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711,...,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711
Bot,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862,...,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862
Brute Force -Web,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
Brute Force -XSS,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
DDOS attack-HOIC,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617,...,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617
DDOS attack-LOIC-UDP,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
DDoS attacks-LOIC-HTTP,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146,...,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146
DoS attacks-GoldenEye,554,554,554,554,554,554,554,554,554,554,...,554,554,554,554,554,554,554,554,554,554
DoS attacks-Hulk,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653,...,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653


In [15]:
##ADDED
test_data.groupby(by="Attack").count()
##

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,8075,8075,8075,8075,8075,8075,8075,8075,8075,8075,...,8075,8075,8075,8075,8075,8075,8075,8075,8075,8075
Brute Force -Web,4455,4455,4455,4455,4455,4455,4455,4455,4455,4455,...,4455,4455,4455,4455,4455,4455,4455,4455,4455,4455
Brute Force -XSS,410,410,410,410,410,410,410,410,410,410,...,410,410,410,410,410,410,410,410,410,410
DoS attacks-Hulk,3995,3995,3995,3995,3995,3995,3995,3995,3995,3995,...,3995,3995,3995,3995,3995,3995,3995,3995,3995,3995
DoS attacks-SlowHTTPTest,1954,1954,1954,1954,1954,1954,1954,1954,1954,1954,...,1954,1954,1954,1954,1954,1954,1954,1954,1954,1954
DoS attacks-Slowloris,1946,1946,1946,1946,1946,1946,1946,1946,1946,1946,...,1946,1946,1946,1946,1946,1946,1946,1946,1946,1946
FTP-BruteForce,544,544,544,544,544,544,544,544,544,544,...,544,544,544,544,544,544,544,544,544,544
Infilteration,4996,4996,4996,4996,4996,4996,4996,4996,4996,4996,...,4996,4996,4996,4996,4996,4996,4996,4996,4996,4996
SQL Injection,654,654,654,654,654,654,654,654,654,654,...,654,654,654,654,654,654,654,654,654,654


In [16]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
##ADDED
X_train, y_train = X, y
X_test = test_data.drop(columns=["Attack", "Label"])
y_test = test_data[["Attack", "Label"]]
##

In [17]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [18]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [19]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [20]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
14549820,172.31.67.20,172.31.0.2,4.716236e-09,3.726003e-09,0.000034,4.275410e-07,0.000070,4.275410e-07,4.738929e-09,4.738929e-09,...,0.0,0.0,0.0,5.035451e-08,5.035451e-08,4.251542e-08,3.431984e-09,0.000026,5.109913e-08,"[4.716236267708683e-09, 3.7260034863814994e-09..."
13845100,172.31.66.55,172.31.0.2,5.658012e-09,5.282345e-09,0.000049,5.129157e-07,0.000115,5.129157e-07,5.685235e-09,5.685235e-09,...,0.0,0.0,0.0,6.040970e-08,6.040970e-08,5.260711e-08,4.632277e-09,0.000009,6.130301e-08,"[5.6580115852816764e-09, 5.282345389367056e-09..."
12678473,172.31.64.108,172.31.0.2,6.536759e-09,5.164284e-09,0.000038,5.925768e-07,0.000119,5.925768e-07,6.568210e-09,6.568210e-09,...,0.0,0.0,0.0,6.979194e-08,6.979194e-08,6.077753e-08,5.351717e-09,0.000036,7.082399e-08,"[6.536758516223423e-09, 5.164284323040382e-09,..."
17430018,172.31.64.85,172.31.0.2,1.209641e-08,9.556615e-09,0.000069,1.096576e-06,0.000104,1.096576e-06,1.215461e-08,1.215461e-08,...,0.0,0.0,0.0,1.291514e-07,1.291514e-07,1.108168e-07,9.903463e-09,0.000066,1.310613e-07,"[1.209640656864491e-08, 9.556614743000617e-09,..."
9445393,172.31.66.17,172.31.0.2,1.020752e-08,8.064322e-09,0.000054,9.253422e-07,0.000113,9.253422e-07,1.025663e-08,1.025663e-08,...,0.0,0.0,0.0,1.089841e-07,1.089841e-07,9.620624e-08,7.427965e-09,0.000056,1.105957e-07,"[1.0207518382181606e-08, 8.064322251987139e-09..."


In [21]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [22]:
# Training graph

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [23]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [24]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [25]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [26]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [27]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [28]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [29]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [30]:
# Convert to GPU
train_g = train_g

In [31]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h']
# edge_features = train_g.edata['h']

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi_CSE.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [32]:
dgi.load_state_dict(torch.load('best_dgi_CSE.pkl'))

<All keys matched successfully>

In [33]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [34]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [35]:
# Convert to GPU
test_g = test_g

In [36]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [37]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [38]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,0.018512,-0.014702,0.014286,-0.001076,0.020005,0.052812,0.075947,0.104963,0.047158,0.062705,...,0.143688,0.099559,-0.014354,0.190111,-0.032931,-0.018382,0.052655,-0.012495,Benign,0
1,0.018512,-0.014702,0.014286,-0.001076,0.020005,0.052812,0.075947,0.104963,0.047158,0.062705,...,0.143688,0.099559,-0.014354,0.190111,-0.032931,-0.018382,0.052655,-0.012495,Benign,0
2,0.018512,-0.014702,0.014286,-0.001076,0.020005,0.052812,0.075947,0.104963,0.047158,0.062705,...,0.143688,0.099559,-0.014354,0.190111,-0.032931,-0.018382,0.052655,-0.012495,Benign,0
3,0.018512,-0.014702,0.014286,-0.001076,0.020005,0.052812,0.075947,0.104963,0.047158,0.062705,...,0.143688,0.099559,-0.014354,0.190111,-0.032931,-0.018382,0.052655,-0.012495,Benign,0
4,0.018512,-0.014702,0.014286,-0.001076,0.020005,0.052812,0.075947,0.104963,0.047158,0.062705,...,0.143688,0.099559,-0.014354,0.190111,-0.032931,-0.018382,0.052655,-0.012495,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754753,0.130016,-0.124823,-0.065747,-0.012904,-0.083787,-0.004330,-0.002197,0.233096,0.060804,0.158346,...,0.179214,-0.005284,0.179867,0.220230,-0.047559,-0.047047,0.067227,0.058269,SSH-Bruteforce,1
754754,0.130016,-0.124823,-0.065747,-0.012904,-0.083787,-0.004330,-0.002197,0.233096,0.060804,0.158346,...,0.179214,-0.005284,0.179867,0.220230,-0.047559,-0.047047,0.067227,0.058269,SSH-Bruteforce,1
754755,0.130016,-0.124823,-0.065747,-0.012904,-0.083787,-0.004330,-0.002197,0.233096,0.060804,0.158346,...,0.179214,-0.005284,0.179867,0.220230,-0.047559,-0.047047,0.067227,0.058269,SSH-Bruteforce,1
754756,0.130016,-0.124823,-0.065747,-0.012904,-0.083787,-0.004330,-0.002197,0.233096,0.060804,0.158346,...,0.179214,-0.005284,0.179867,0.220230,-0.047559,-0.047047,0.067227,0.058269,SSH-Bruteforce,1


# Embeddings CBLOF  Embeddings

In [39]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [40]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [41]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [42]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [06:44<00:00, 11.23s/it]

{'n_estimators': 10, 'con': 0.1}
0.4165801757810793
              precision    recall  f1-score   support

           0     0.1917    0.2162    0.2032     16150
           1     0.6480    0.6128    0.6299     38022

    accuracy                         0.4946     54172
   macro avg     0.4199    0.4145    0.4166     54172
weighted avg     0.5120    0.4946    0.5027     54172



In [43]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [04:07<00:00,  6.87s/it]

{'n_estimators': 7, 'con': 0.2}
0.3972731127938236
              precision    recall  f1-score   support

           0     0.2352    0.4324    0.3047     16150
           1     0.6255    0.4026    0.4899     38022

    accuracy                         0.4115     54172
   macro avg     0.4303    0.4175    0.3973     54172
weighted avg     0.5091    0.4115    0.4347     54172



In [44]:
###  CBLOF RAW

In [45]:
df_raw_train = pd.concat([X_train.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_train], axis=1)
df_raw_test = pd.concat([X_test.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_test], axis=1)

In [46]:
raw_benign_train_samples = df_raw_train[df_raw_train.Label == 0].drop(columns=["Label", "Attack"])
raw_normal_train_samples = df_raw_train.drop(columns=["Label", "Attack"])

raw_train_labels = df_raw_train["Label"]
raw_test_labels = df_raw_test["Label"]

raw_test_samples = df_raw_test.drop(columns=["Label", "Attack"])

In [47]:
from pyod.models.cblof import CBLOF

n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = CBLOF(n_clusters=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  6%|▌         | 2/36 [00:01<00:25,  1.35it/s]

2


 17%|█▋        | 6/36 [00:04<00:18,  1.65it/s]

2


 22%|██▏       | 8/36 [00:05<00:15,  1.77it/s]

3


 33%|███▎      | 12/36 [00:08<00:16,  1.43it/s]

3


100%|██████████| 36/36 [00:28<00:00,  1.28it/s]

{'n_estimators': 10, 'con': 0.04}
0.5667519236537578
              precision    recall  f1-score   support

           0     0.4078    1.0000    0.5794      8075
           1     1.0000    0.3833    0.5541     19011

    accuracy                         0.5671     27086
   macro avg     0.7039    0.6916    0.5668     27086
weighted avg     0.8235    0.5671    0.5617     27086



In [48]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

 28%|██▊       | 10/36 [00:07<00:16,  1.57it/s]

3


 31%|███       | 11/36 [00:07<00:13,  1.80it/s]

3


 33%|███▎      | 12/36 [00:08<00:11,  2.02it/s]

3


100%|██████████| 36/36 [00:30<00:00,  1.20it/s]

benign only
{'n_estimators': 5}
0.48148192960290315
              precision    recall  f1-score   support

           0     0.3678    0.9999    0.5378      8075
           1     0.9998    0.2700    0.4252     19011

    accuracy                         0.4876     27086
   macro avg     0.6838    0.6349    0.4815     27086
weighted avg     0.8114    0.4876    0.4588     27086



In [49]:
# HBOS  Embeddings

In [50]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [51]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [52]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [04:55<00:00,  8.21s/it]

{'n_estimators': 5, 'con': 0.2}
0.43525655387284135
              precision    recall  f1-score   support

           0     0.2094    0.2162    0.2127     16150
           1     0.6624    0.6532    0.6578     38022

    accuracy                         0.5229     54172
   macro avg     0.4359    0.4347    0.4353     54172
weighted avg     0.5273    0.5229    0.5251     54172



In [53]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [05:44<00:00,  9.57s/it]

{'n_estimators': 20, 'con': 0.2}
0.3487063280294605
              precision    recall  f1-score   support

           0     0.2131    0.4324    0.2855     16150
           1     0.5718    0.3219    0.4119     38022

    accuracy                         0.3548     54172
   macro avg     0.3924    0.3772    0.3487     54172
weighted avg     0.4648    0.3548    0.3742     54172



In [54]:
##  HBOS  RAw

In [55]:
from pyod.models.cblof import CBLOF

n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = HBOS(n_bins=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:24<00:00,  1.49it/s]

{'n_estimators': 30, 'con': 0.2}
0.9834003120401145
              precision    recall  f1-score   support

           0     0.9965    0.9573    0.9765      8075
           1     0.9822    0.9986    0.9903     19011

    accuracy                         0.9863     27086
   macro avg     0.9893    0.9779    0.9834     27086
weighted avg     0.9864    0.9863    0.9862     27086



In [56]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = HBOS(n_bins=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:26<00:00,  1.36it/s]

benign only
{'n_estimators': 10}
0.9837045676898976
              precision    recall  f1-score   support

           0     0.9612    0.9939    0.9773      8075
           1     0.9974    0.9830    0.9901     19011

    accuracy                         0.9862     27086
   macro avg     0.9793    0.9884    0.9837     27086
weighted avg     0.9866    0.9862    0.9863     27086



In [57]:
##  PCA  Emb

In [58]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:36<00:00,  4.36s/it]

{'n_estimators': 5, 'con': 0.2}
0.3881358995210988
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000     16150
           1     0.6803    0.9038    0.7763     38022

    accuracy                         0.6343     54172
   macro avg     0.3401    0.4519    0.3881     54172
weighted avg     0.4775    0.6343    0.5448     54172



In [59]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:36<00:00,  4.36s/it]

{'n_estimators': 5, 'con': 0.2}
0.23007166227115183
              precision    recall  f1-score   support

           0     0.2982    1.0000    0.4594     16150
           1     1.0000    0.0004    0.0007     38022

    accuracy                         0.2984     54172
   macro avg     0.6491    0.5002    0.2301     54172
weighted avg     0.7908    0.2984    0.1375     54172



In [60]:
##  PCA  RAw

In [61]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples)
   
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:25<00:00,  1.43it/s]

{'n_estimators': 5, 'con': 0.05}
0.9383921491246581
              precision    recall  f1-score   support

           0     0.8780    0.9555    0.9151      8075
           1     0.9804    0.9436    0.9616     19011

    accuracy                         0.9472     27086
   macro avg     0.9292    0.9496    0.9384     27086
weighted avg     0.9499    0.9472    0.9478     27086



In [62]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples)

    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:25<00:00,  1.40it/s]

benign only
{'n_estimators': 5}
0.9031897718154859
              precision    recall  f1-score   support

           0     0.8991    0.8275    0.8618      8075
           1     0.9291    0.9605    0.9446     19011

    accuracy                         0.9209     27086
   macro avg     0.9141    0.8940    0.9032     27086
weighted avg     0.9202    0.9209    0.9199     27086



In [63]:
##  IF  Emb

In [64]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [01:41<00:00,  4.23s/it]

{'n_estimators': 20, 'con': 0.2}
0.43525655387284135
              precision    recall  f1-score   support

           0     0.2094    0.2162    0.2127     16150
           1     0.6624    0.6532    0.6578     38022

    accuracy                         0.5229     54172
   macro avg     0.4359    0.4347    0.4353     54172
weighted avg     0.5273    0.5229    0.5251     54172



In [65]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [01:17<00:00,  3.24s/it]

{'n_estimators': 20, 'con': 0.2}
0.3518564021638647
              precision    recall  f1-score   support

           0     0.2760    0.7162    0.3984     16150
           1     0.6261    0.2019    0.3053     38022

    accuracy                         0.3552     54172
   macro avg     0.4510    0.4590    0.3519     54172
weighted avg     0.5217    0.3552    0.3331     54172



In [66]:
##  IF  Raw

In [67]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples.to_numpy())
   
    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:24<00:00,  1.01s/it]

{'n_estimators': 100, 'con': 0.05}
0.879285281225455
              precision    recall  f1-score   support

           0     0.8809    0.7782    0.8264      8075
           1     0.9102    0.9553    0.9322     19011

    accuracy                         0.9025     27086
   macro avg     0.8955    0.8667    0.8793     27086
weighted avg     0.9015    0.9025    0.9007     27086



In [68]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples.to_numpy())

    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:25<00:00,  1.05s/it]

benign only
{'n_estimators': 150}
0.9160768684394578
              precision    recall  f1-score   support

           0     0.8341    0.9437    0.8855      8075
           1     0.9747    0.9203    0.9467     19011

    accuracy                         0.9272     27086
   macro avg     0.9044    0.9320    0.9161     27086
weighted avg     0.9327    0.9272    0.9284     27086

